In [1]:
import requests
import pandas as pd
from funcoes import *

In [2]:
r = requests.get("https://mof.tech.northwestern.edu/mofs.json")

print(r)

<Response [200]>


In [3]:
data = r.json()['results']

In [4]:
df = pd.DataFrame.from_dict(data)

In [5]:
df = funcao_extrair_features_isotherms(df)

In [6]:
df_mof = df.iloc[100:130,:]
df_mof = extrair_cif(df_mof,minimo=3)

GlobalSymmetryFeatures:   0%|          | 0/30 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/30 [00:00<?, ?it/s]

In [24]:
#print(df_mof)

In [7]:
def importar_dados(dataframe):
    'importar os dados provenientes do site das MOFs'
    dados_hmof = pd.read_excel(dataframe)
    return dados_hmof

In [8]:
dataframe = ('Pasta1.xlsx')
df_import = importar_dados(dataframe)

## Importações



In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Divisão treino-teste

In [10]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ['lcd', 'pld', 'void_fraction', 'surface_area_m2g', 'surface_area_m2cm3', 'CO2_0.01', 'CO2_0.1', 'CO2_2.5', 'CO2_0.05', 'CO2_0.5',
            'N2_0.09', 'N2_0.9', 'CH4_2.5', 'CH4_4.5', 'CH4_0.05', 'CH4_0.5', 'CH4_0.9', 'CH4_35', 'H2_2', 'H2_100']
TARGET = ['Xe_1']

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [11]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [12]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [13]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [14]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

In [15]:
y_prev = minha_MLP(X_treino)

In [16]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [17]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=20, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [18]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0367)
1 tensor(0.0333)
2 tensor(0.0305)
3 tensor(0.0284)
4 tensor(0.0268)
5 tensor(0.0257)
6 tensor(0.0250)
7 tensor(0.0244)
8 tensor(0.0239)
9 tensor(0.0232)
10 tensor(0.0225)
11 tensor(0.0216)
12 tensor(0.0207)
13 tensor(0.0197)
14 tensor(0.0188)
15 tensor(0.0179)
16 tensor(0.0171)
17 tensor(0.0163)
18 tensor(0.0156)
19 tensor(0.0150)
20 tensor(0.0143)
21 tensor(0.0137)
22 tensor(0.0130)
23 tensor(0.0123)
24 tensor(0.0115)
25 tensor(0.0108)
26 tensor(0.0100)
27 tensor(0.0093)
28 tensor(0.0086)
29 tensor(0.0080)
30 tensor(0.0073)
31 tensor(0.0068)
32 tensor(0.0062)
33 tensor(0.0057)
34 tensor(0.0052)
35 tensor(0.0048)
36 tensor(0.0045)
37 tensor(0.0041)
38 tensor(0.0037)
39 tensor(0.0034)
40 tensor(0.0031)
41 tensor(0.0029)
42 tensor(0.0027)
43 tensor(0.0025)
44 tensor(0.0023)
45 tensor(0.0022)
46 tensor(0.0021)
47 tensor(0.0020)
48 tensor(0.0020)
49 tensor(0.0019)
50 tensor(0.0019)
51 tensor(0.0018)
52 tensor(0.0018)
53 tensor(0.0018)
54 tensor(0.0017)
55 tensor(0.0017)
56

In [19]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.57337847] [0.54884891]
[1.553999] [1.56895988]
[0.72190402] [0.73024642]
[2.27605493] [2.26404143]
[3.03556895] [2.90249191]
[0.78030701] [0.83332714]
[2.368495] [2.3748884]
[2.65115801] [2.62601024]
[1.70751903] [1.72794787]
[0.95890204] [0.99780764]
[0.760066] [0.75703061]
[0.75630199] [0.75636953]
[4.25403996] [4.27448824]
[1.8873291] [1.93577568]
[2.61967902] [2.68150391]
[2.15343697] [2.17377576]
[0.63327298] [0.66106856]
[3.86579008] [3.90085469]
[2.45340706] [2.39573878]
[0.3991435] [0.3890996]
[1.692429] [1.65433775]
[0.78046498] [0.88158088]
[1.23384097] [1.26153532]
[0.937126] [0.93565333]
[1.71586797] [1.73613322]
[0.93059903] [0.9005777]
[1.02393604] [1.0231407]
[2.59136392] [2.5960213]
[0.65360399] [0.57503988]
[1.03309998] [1.03154481]
[0.931082] [0.93880285]
[3.46831008] [3.5177509]
[3.31250995] [3.2600264]
[0.60076829] [0.62959012]
[1.99398297] [2.00544515]
[4.56051013] [4.80785457]
[0.49076277] [0.47841784]
[0.64449702] [0.64852923]
[2.51258008] [2.62545568]
[3.2692

In [20]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=20, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [21]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.74338086] [0.43965401]
[1.09562605] [1.13148071]
[0.00526056] [0.0294201]
[0.93107099] [0.95399534]
[0.96158997] [0.93476794]
[3.89166992] [3.81543407]
[3.0270899] [3.02538887]
[1.13198603] [0.94595532]
[0.91508726] [0.99972312]
[2.70047164e-13] [0.01950079]
[0.74763] [0.71401554]
[0.992004] [1.03487973]
[2.39291091] [2.63439297]
[1.82485507] [1.96241415]
[1.93217601] [2.08164213]
[0.0331167] [0.26365543]
[0.70727699] [0.64464705]
[1.27770099] [1.33877415]
[4.21562986] [4.37085751]
[0.77424353] [0.58402555]


In [22]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 0.13545357791685939
